### Elementy sztucznej inteligencji
# Projekt: Binarne Drzewa Decyzyjne
autorzy:
1. Arkadiusz Florek
2. Maciej Komosa
3. Albert Pieniądz
4. Jakub Zięba

In [1]:
import numpy as np, pandas as pd

Funkcje do obliczania entropii dla danej konkluzji

In [2]:
from math import log2

class Wezel:

    def __init__(self):
        self.wartosc = None
        self.kolejny = None
        self.dzieci = None

class DrzewoBinarneID3:

    def __init__(self, dane):
        #self.dane to nie są dane, w obrębie klasy korzystamy tylko z self.danych
        self.dane = dane[1:, :]
        #nagłówki
        self.nazwy_przeslanek = dane[0, :]
        #self.explanatory hehe
        self.klasy_konkluzji = np.unique(self.dane[:, -1])
        #jestem prawie pewny że połowa z tych rzeczy jest niepotrzebna
        self.ilosc_wierszy_dla_danej_klasy_konkluzji = [list(self.dane[:, -1]).count(x) for x in self.klasy_konkluzji]
        #entropia całego układu, czyli właściwie to klas
        self.entropia = self._entropia(self.dane[:, -1])
        #teoretycznie w tym węźle ma się mieścić całe drzewo, na zasadzie zagnieżdżonych referencji
        #byłoby jeszcze zabawniej jakbym rozumiał jak to zrobić 
        self.wezel = self._id3(self.dane, None)
 
    def _entropia(self, dane):
        # Tworzymy słownik, w którym kluczem jest wartość, a wartością liczba wystąpień tej wartości
        wystapienia_wartosci = {}
        for wiersz in dane:
            atrybut = wiersz[-1]
            if atrybut not in wystapienia_wartosci:
                wystapienia_wartosci[atrybut] = 0
            wystapienia_wartosci[atrybut] += 1
    
        # Obliczamy entropię dla danej tabeli
        entropia = 0.0
        ilosc_wierszy = len(dane)
        for wartosc in wystapienia_wartosci.values():
            prawdopodobienstwo = wartosc / ilosc_wierszy
            entropia -= prawdopodobienstwo * log2(prawdopodobienstwo)
    
        return entropia
 
    def _zysk_informacji(self, dane, przeslanka):
        # Tworzymy słownik, w którym kluczem jest wartość, a wartością lista wierszy z tą wartością
        wartosci_przeslanek = {}
        for wiersz in dane:
            wartosc = wiersz[przeslanka]
            if wartosc not in wartosci_przeslanek:
                wartosci_przeslanek[wartosc] = []
            wartosci_przeslanek[wartosc].append(wiersz)
    
        # Obliczamy entropię dla każdego podziału i sumujemy, aby uzyskać entropię po podziale
        ilosc_wierszy = len(dane)
        entropie = []
        for wartosci_przeslanki in wartosci_przeslanek.values():
            ilosc_wartosci_przeslanki = len(wartosci_przeslanki)
            entropia_przeslanki = self._entropia(wartosci_przeslanki)
            entropie.append((ilosc_wartosci_przeslanki / ilosc_wierszy) * entropia_przeslanki)
        entropia_przeslanki = sum(entropie)
    
        # Obliczamy przyrost informacji
        zysk = self._entropia(dane) - entropia_przeslanki
    
        return zysk

    def _przeslanka_o_najwiekszym_zysku(self, dane):
        
        # tabela przeslanek, czyli tabela o wartosciach
        # z zakresu dlugosci jednego wiersza odjąć konkluzję
        przeslanki = [*range(dane[0].size - 1)]

        entropie_przeslanek = [self._zysk_informacji(dane, przeslanka) for przeslanka in przeslanki]
        przeslanka_o_najwiekszym_zysku = przeslanki[entropie_przeslanek.index(max(entropie_przeslanek))]

        return przeslanka_o_najwiekszym_zysku
    
    #szczerze to nie wiem jak ten sposób dzielenia na węzły zrobić, ale ma być w tej funkcji i bazować na funkcji _przeslanka_o_najwiekszym_zysku
    def _id3(self, dane, wezel):
        
        if not wezel:
            wezel = Wezel()
        
        if len(dane) == 0:
            wezel.wartosc = max()
        
        

Klasa węzła

In [3]:
#pobieranie danych z nagłówkami i przesyłanie ich do funkcji - dane i nagłówki oddzielnie - trochę śmie
dane = pd.read_csv("sport.csv", header=None).to_numpy()
bdd = DrzewoBinarneID3(dane)

AttributeError: 'DrzewoBinarneID3' object has no attribute 'wezel'

Stare rzeczy, przed uporządkowaniem do klas

In [65]:
dane = pd.read_csv("sport.csv").to_numpy()

print(entropia(dane))

zyski_informacji_przeslanek = []
dtab = []

przeslanki = np.array(range(dane[0].size))
print(przeslanka_o_najwiekszym_zysku(dane))

while przeslanki.size > 1:
    for i in przeslanki:
        if i == przeslanki[-1]:
            continue
        zyski_informacji_przeslanek.append(zysk_informacji(dane, i))
    imax = np.argmax(zyski_informacji_przeslanek)
    #print("przed usunieciem: przeslanki ", przeslanki, ", zysk", zyski_informacji_przeslanek, ", imax ", imax)
    #print("przed", przeslanki, dane[0])
    przeslanki = np.delete(przeslanki, imax)
    dane = np.delete(dane, imax, 1)
    #print("po", przeslanki, dane[0])
    #print("po usunieciu: przeslanki ", przeslanki)
    dtab.append(zyski_informacji_przeslanek.copy())
    zyski_informacji_przeslanek.clear()

dtab


2.67943596719348
4


[[0.1651500355641029,
  0.06226147757579792,
  0.09413277111930896,
  0.06929896561476978,
  0.21714308177218555],
 [0.1651500355641029,
  0.06226147757579792,
  0.09413277111930896,
  0.06929896561476978],
 [0.09413277111930896, 0.06929896561476978, 2.67943596719348],
 [0.09413277111930896, 2.67943596719348],
 [2.67943596719348]]